In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from Bio import Entrez, SeqIO

In [6]:
sacc_id = 'GCF_000146045.2_R64'
yl_id = 'GCF_000002525.2_ASM252v1'
ids = [sacc_id, yl_id]


In [11]:
# Define a function to make blast database for either protein of nucleotide
def make_blast_db(id,folder='prot',db_type='prot'):
    import os
    out_file ='%s/%s.fa.pin'%(folder, id)
    files =glob('%s/*.fa.pin'%folder)
    
    if out_file in files:
        print (id, 'already has a blast db')
        return
    if db_type=='nucl':
        ext='fna'
    else:
        ext='faa'

    cmd_line='makeblastdb -in %s/%s.%s -dbtype %s' %(folder, id, ext, db_type)
    
    print ('making blast db with following command line...')
    print (cmd_line)
    os.system(cmd_line)

In [12]:
for seq in ids:
    new_seq = seq+'_protein'
    make_blast_db(new_seq,folder='prot',db_type='prot')

making blast db with following command line...
makeblastdb -in prot/GCF_000146045.2_R64_protein.faa -dbtype prot


Building a new DB, current time: 10/14/2022 12:03:08
New DB name:   /home/kkrishnan/SBRG/Sequencing/Y-Lipolytica/blast/prot/GCF_000146045.2_R64_protein.faa
New DB title:  prot/GCF_000146045.2_R64_protein.faa
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 6016 sequences in 0.109475 seconds.
making blast db with following command line...
makeblastdb -in prot/GCF_000002525.2_ASM252v1_protein.faa -dbtype prot


Building a new DB, current time: 10/14/2022 12:03:08
New DB name:   /home/kkrishnan/SBRG/Sequencing/Y-Lipolytica/blast/prot/GCF_000002525.2_ASM252v1_protein.faa
New DB title:  prot/GCF_000002525.2_ASM252v1_protein.faa
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 6472 sequences in 0.115157 seconds.


In [13]:
# define a function to run BLASTp
def run_blastp(seq,db,in_folder='prot', out_folder='bbh', out=None,outfmt=6,evalue=0.001,threads=1):
    import os
    if out==None:
        out='%s/%s_vs_%s.txt'%(out_folder, seq, db)
        print(out)
    
    files =glob('%s/*.txt'%out_folder)
#    if out in files:
#        print (seq, 'already blasted')
#        return
    
    print ('blasting %s vs %s'%(seq, db))
    
    db = '%s/%s.faa'%(in_folder, db)
    seq = '%s/%s.faa'%(in_folder, seq)
    cmd_line='blastp -db %s -query %s -out %s -evalue %s -outfmt %s -num_threads %i' \
    %(db, seq, out, evalue, outfmt, threads)
    
    print ('running blastp with following command line...')
    print (cmd_line)
    os.system(cmd_line)
    return out

In [14]:
# define a function to get sequence length 

def get_gene_lens(query, in_folder='prot'):
    file = '/home/kkrishnan/SBRG/Sequencing/Y-Lipolytica/blast/prot/%s/%s.faa'%(in_folder, query)
    file = '%s/%s.faa'%(in_folder, query)
    handle = open(file)
    records = SeqIO.parse(handle, "fasta")
    out = []
    
    for record in records:
        out.append({'gene':record.name, 'gene_length':len(record.seq)})
    
    out = pd.DataFrame(out)
    return out

In [17]:
# define a function to get Bi-Directional BLASTp Best Hits
def get_bbh(query, subject, in_folder='bbh'):    
    
    #Utilize the defined protein BLAST function
    run_blastp(query, subject)
    run_blastp(subject, query)
    
    query_lengths = get_gene_lens(query, in_folder='prot')
    subject_lengths = get_gene_lens(subject, in_folder='prot')
    
    #Define the output file of this BLAST
    out_file = '%s/%s_vs_%s_parsed.csv'%(in_folder,query, subject)
    files=glob('%s/*_parsed.csv'%in_folder)
    
    #Combine the results of the protein BLAST into a dataframe
    print ('parsing BBHs for', query, subject)
    cols = ['gene', 'subject', 'PID', 'alnLength', 'mismatchCount', 'gapOpenCount', 'queryStart', 'queryEnd', 'subjectStart', 'subjectEnd', 'eVal', 'bitScore']
    bbh=pd.read_csv('%s/%s_vs_%s.txt'%(in_folder,query, subject), sep='\t', names=cols)
    bbh = pd.merge(bbh, query_lengths) 
    bbh['COV'] = bbh['alnLength']/bbh['gene_length']
    
    bbh2=pd.read_csv('%s/%s_vs_%s.txt'%(in_folder,subject, query), sep='\t', names=cols)
    bbh2 = pd.merge(bbh2, subject_lengths) 
    bbh2['COV'] = bbh2['alnLength']/bbh2['gene_length']
    out = pd.DataFrame()
    
    # Filter the genes based on coverage
    bbh = bbh[bbh.COV>=0.25]
    bbh2 = bbh2[bbh2.COV>=0.25]
    
    #Delineate the best hits from the BLAST
    for g in bbh.gene.unique():
        res = bbh[bbh.gene==g]
        if len(res)==0:
            continue
        best_hit = res.loc[res.PID.idxmax()]
        best_gene = best_hit.subject
        res2 = bbh2[bbh2.gene==best_gene]
        if len(res2)==0:
            continue
        best_hit2 = res2.loc[res2.PID.idxmax()]
        best_gene2 = best_hit2.subject
        if g==best_gene2:
            best_hit['BBH'] = '<=>'
        else:
            best_hit['BBH'] = '->'
        out=pd.concat([out, pd.DataFrame(best_hit).transpose()])
    
    #Save the final file to a designated CSV file    
    out.to_csv(out_file)

In [18]:
get_bbh('GCF_000146045.2_R64_protein','GCF_000002525.2_ASM252v1_protein', in_folder='bbh')

bbh/GCF_000146045.2_R64_protein_vs_GCF_000002525.2_ASM252v1_protein.txt
blasting GCF_000146045.2_R64_protein vs GCF_000002525.2_ASM252v1_protein
running blastp with following command line...
blastp -db prot/GCF_000002525.2_ASM252v1_protein.faa -query prot/GCF_000146045.2_R64_protein.faa -out bbh/GCF_000146045.2_R64_protein_vs_GCF_000002525.2_ASM252v1_protein.txt -evalue 0.001 -outfmt 6 -num_threads 1
bbh/GCF_000002525.2_ASM252v1_protein_vs_GCF_000146045.2_R64_protein.txt
blasting GCF_000002525.2_ASM252v1_protein vs GCF_000146045.2_R64_protein
running blastp with following command line...
blastp -db prot/GCF_000146045.2_R64_protein.faa -query prot/GCF_000002525.2_ASM252v1_protein.faa -out bbh/GCF_000002525.2_ASM252v1_protein_vs_GCF_000146045.2_R64_protein.txt -evalue 0.001 -outfmt 6 -num_threads 1
parsing BBHs for GCF_000146045.2_R64_protein GCF_000002525.2_ASM252v1_protein


/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

...

/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

/tmp/ipykernel_1706353/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_1706353/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_1706353/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_1706353/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

/tmp/ipykernel_1706353/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

/tmp/ipykernel_1706353/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
